In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier


import pandas as pd
import numpy as np
import os
import glob

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

Using TensorFlow backend.


In [2]:
personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SimpleAutoencoder'] = {
    'regularizer' :['regularizers.l1(10e-5)', 'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
    'encoding_dim': [40],
    'activation': ['relu'],
    'optimizer': ['adadelta', "SGD"],
    'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
       'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
    'epochs':[100],
    'batch_size':[200]
}

In [3]:
#one hot encoding function

#function that takes in a features pandas dataframe and turns it into a numpy array 
#with only the categorical variables one-hot encoded (and leaving out one of the features 
#of every one-hot encoded variable as baseline)
#@param:  features is a pandas df of features; threshold number is the number of unique values 
#a variable should have in order to be considered categorical
#@return:  a numpy matrix containing the original dataset but with the categorical variables
#one hot encoded and one of the one hot encoded features per cat variable left out; 
#for example an original feature set of 10 categorical variables with three categories each will 
#be transoformed into a numpy array with 20 dichotomous variables

def one_hot_encode(features, cat_threshold_number = 5):
    num_unique_vals_dict = {}
    feature_names = list(features)
    for feature in feature_names:
        label_encoder = LabelEncoder()
        features.loc[:, feature] = label_encoder.fit_transform(features.loc[:, feature])
        num_unique_vals_dict[feature] = len(label_encoder.classes_)

    features_to_onehot = []
    for feature in num_unique_vals_dict:
        if num_unique_vals_dict[feature] <= cat_threshold_number and num_unique_vals_dict[feature] > 1:
            features_to_onehot = features_to_onehot + [feature]

    #create index array listing indices in orginal feature names array that are present in features_to_onehot
    indices_to_onehot = np.nonzero(np.in1d(feature_names, features_to_onehot))

    onehot_encoder = OneHotEncoder(categorical_features = indices_to_onehot, sparse = False)
    features = onehot_encoder.fit_transform(features)

    idx_to_delete = np.cumsum([0] + list(num_unique_vals_dict.values()))

    idx_to_keep = [i for i in range(features.shape[1]) if i not in idx_to_delete]

    features = features[:, idx_to_keep]

    return features

In [4]:
#read in data
data = pd.read_table("sample_data.txt")
data = data.iloc[:, :11]
dv = data.iloc[:, 10:11]
features = data.iloc[:, :10]

In [5]:
#one hot encode data, then split it into training and validation
X = one_hot_encode(features)
y = dv
y = pd.np.array(y).ravel()
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.33, random_state = 42)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [6]:
x_train.shape

(670, 20)

In [7]:
y_train.shape

(670,)

In [8]:
from tpot.builtins import SparseAutoencoder

ImportError: cannot import name 'SparseAutoencoder'

In [9]:
sae = SparseAutoencoder(encoding_dim=10, 
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=100, batch_size=200)

NameError: name 'SparseAutoencoder' is not defined

In [16]:
sae.fit(x_train, y_train)

NameError: name 'sae' is not defined

In [17]:
sae.transform(x_train).shape

NameError: name 'sae' is not defined

In [18]:
x_train.shape

(670, 20)

In [19]:
tpot = TPOTClassifier(generations=10, config_dict=personal_config,
                        population_size=10, verbosity=3,
                        template = 'SparseAutoencoder-Classifier')

TypeError: __init__() got an unexpected keyword argument 'template'